In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
db = '../data/database/NYC_CRIME_DATABASE.db'
conn = sql.connect(db)

In [4]:
# q = '''SELECT ARREST_KEY FROM ARRESTS;'''
# pd.read_sql_query(q, conn)

In [3]:
q = '''SELECT CMPLNT_KEY FROM COMPLAINTS;'''
pd.read_sql_query(q, conn)

,CMPLNT_KEY
0,25636218
1,23364313
2,25523851
3,25435616
4,34143267
...,...
8858840,272431385
8858841,273033152
8858842,270041815
8858843,273151416


### Ratio of Complaints to Arrests per Borough

In [ ]:
q = '''
SELECT
    A.ARREST_KEY,
    C.CMPLNT_KEY,
    A.BORO
FROM ARRESTS A, COMPLAINTS C
WHERE A.BORO = C.BORO;
'''

pd.read_sql_query(q, conn)

In [ ]:
GROUP BY A.BORO

In [ ]:
conn.close()